In [1]:
import os, sys
sys.path.insert(1, os.path.abspath("../../../"))  # for dowhy source code
sys.path.insert(2, os.path.abspath("../../../../EconML/")) # for econml source code

In [2]:
import numpy as np
import pandas as pd
import logging

import dowhy
from dowhy import CausalModel
import dowhy.datasets

In [3]:
data = dowhy.datasets.linear_dataset(10, num_common_causes=4, num_samples=10000,
                                    num_instruments=0, num_effect_modifiers=2,
                                    treatment_is_binary=False)
df=data['df']
df.head()

,X0,X1,W0,W1,W2,W3,v,y
0,-0.355812,0.081412,0.795749,-0.131001,-2.262737,0.684836,-3.718250,-45.475559
1,1.465739,-1.517709,-0.727935,-1.878542,0.451740,-0.428575,-10.526795,-109.481331
2,0.830387,-0.705246,0.810064,-0.639895,-2.353308,-0.303040,-8.222476,-94.187557
3,0.200510,-0.184273,-1.055428,0.120913,0.908116,-0.793236,-2.417047,-22.612623
4,0.183300,-1.356460,-0.876159,-1.810650,-1.736931,-0.807156,-19.792463,-213.371791


In [4]:
model = CausalModel(data=data["df"], 
                    treatment=data["treatment_name"], outcome=data["outcome_name"], 
                    graph=data["gml_graph"])

INFO:dowhy.causal_model:Model to find the causal effect of treatment ['v'] on outcome ['y']


In [5]:
identified_estimand= model.identify_effect()
print(identified_estimand)

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['W3', 'W1', 'W2', 'Unobserved Confounders', 'W0']


WARN: Do you want to continue by ignoring these unobserved confounders? [y/n] y


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]


Estimand type: ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
d                             
──(Expectation(y|W3,W1,W2,W0))
dv                            
Estimand assumption 1, Unconfoundedness: If U→v and U→y then P(y|v,W3,W1,W2,W0,U) = P(y|v,W3,W1,W2,W0)
### Estimand : 2
Estimand name: iv
No such variable found!



In [14]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.ensemble import GradientBoostingRegressor
estimate = model.estimate_effect(identified_estimand, method_name="backdoor.econml.dml.DMLCate",
                                method_params={"init_params":{'model_y':GradientBoostingRegressor(),'model_t': GradientBoostingRegressor(),"model_final":LassoCV(), 'featurizer':PolynomialFeatures(degree=1, include_bias=True)}, "fit_params":{}})

INFO:dowhy.causal_estimator:INFO: Using EconML Estimator
INFO:dowhy.causal_estimator:b: y~v+W3+W1+W2+W0
/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/gradient_boosting.py:145

TypeError: unsupported operand type(s) for /: 'DMLCateEstimator' and 'float'

In [ ]:
import econml
from econml.dml import DMLCateEstimator, LinearDMLCateEstimator, SparseLinearDMLCateEstimator
from sklearn.preprocessing import PolynomialFeatures
from econml.bootstrap import BootstrapEstimator
from sklearn.linear_model import MultiTaskElasticNet

import numpy as np

# Instance parameters
n_controls = 2
n_instruments = 1
n_features = 1
n_treatments = 1
alpha = np.random.normal(size=(n_controls, 1))
beta = np.random.normal(size=(n_instruments, 1))
gamma = np.random.normal(size=(n_treatments, 1))
delta = np.random.normal(size=(n_treatments, 1))
zeta = np.random.normal(size=(n_controls, 1))

In [ ]:
#Generating Data
n_samples = 10000
W = np.random.normal(size=(n_samples, n_controls)) # confounders
Z = np.random.normal(size=(n_samples, n_instruments)) # instrumental variables
X = np.random.normal(size=(n_samples, n_features)) # effect modifiers
eta = np.random.normal(size=(n_samples, n_treatments)) 
epsilon = np.random.normal(size=(n_samples, 1))
T = np.dot(W, alpha) + np.dot(Z, beta) + eta # treatment
y = np.dot(T**2, gamma) + np.dot(np.multiply(T, X), delta) + np.dot(W, zeta) + epsilon
print(gamma, delta) # causal effect parameters

In [ ]:
# Fit counterfactual model
est = LinearDMLCateEstimator()
    #model_y=MultiTaskElasticNet(alpha=0.1),
    #                model_t=MultiTaskElasticNet(alpha=0.1),
    #                featurizer=PolynomialFeatures(degree=1, include_bias=False))
print(W, W.shape)
print(np.ndarray(shape=(n_samples,2), buffer=np.array(df[['X0','X1']])), df['X0'].shape)
est.fit(
    np.ndarray(shape=(n_samples,1), buffer=np.array(df['y'])),
    np.ndarray(shape=(n_samples,1), buffer=np.array(df['v'])),
    X=np.ndarray(shape=(n_samples,1), buffer=np.array(df[['X0', 'X1']])),
    W=np.ndarray(shape=(n_samples,1), buffer=np.array(df['W0'])),)

In [ ]:
X_test = X
# Estimate heterogeneous treatment effects from going from treatment 0 to treatment 1
T0_test = np.zeros((X_test.shape[0], n_treatments))
T1_test = np.ones((X_test.shape[0], n_treatments))
#hetero_te = cfest.effect(T0_test, T1_test, X_test)
te_pred = est.const_marginal_effect(np.array([[np.median(X)]]))
print(te_pred)

In [ ]:
X_test = X
# Estimate heterogeneous treatment effects from going from treatment 0 to treatment 1
T0_test = np.zeros((X_test.shape[0], n_treatments))
T1_test = np.ones((X_test.shape[0], n_treatments))
print(est.effect(X, T0=T0_test, T1=T1_test)[:5])
# To get the coefficients of the polynomial fitted in the final stage we can
# access the coef_ attribute of the fitted second stage model. This would
# return the coefficients in front of each term in the vector T⊗ϕ(X).
a_hat = est.coef_
y0 = np.dot(T0_test**2, gamma) + np.dot(np.multiply(T0_test, X), delta) + np.dot(W, zeta) + epsilon
y1 = np.dot(T1_test**2, gamma) + np.dot(np.multiply(T1_test, X), delta) + np.dot(W, zeta) + epsilon
tau = y1-y0
print(tau[:5])
print(a_hat) 
print(gamma, delta)

In [ ]:
# Estimate heterogeneous marginal effects around treatment 0
T_test = np.zeros((X_test.shape[0], n_treatments))
hetero_marginal_te = est.marginal_effect(T_test, X_test)
print(y[1])
hetero_marginal_te[1]

In [ ]:
# Estimate average treatment effects over a population of z's
T0_test = np.zeros((X_test.shape[0], n_treatments))
T1_test = np.ones((X_test.shape[0], n_treatments))

# average treatment effect
ate = np.mean(est.effect(X_test, T0=T0_test, T1=T1_test)) # returns estimate of γ + δ 𝔼[x]
np.ndarray(shape=(10,1), buffer=X_test[[X_test>1]])
# average treatment effect of population with x>1/2
# returns estimate of γ + δ 𝔼[x | x>1/2]
#cate = np.mean(est.effect(X_test[X_test>1/2], T0=T0_test[X_test>1/2], T1=T1_test[X_test>1/2] ))

In [ ]:
# Estimate expected lift of treatment policy: π(z) = 𝟙{x > 0} over existing policy
Pi0_test = T
Pi1_test = (X_test > 0) * 1.
# returns estimate of γ/2 + δ/√(2π)
policy_effect = np.mean(est.effect(Pi0_test, Pi1_test, X_test))

# Estimate expected lift of treatment policy: π(x) = 𝟙{x > 0} over baseline of no treatment
Pi0_test = np.zeros((X_test.shape[0], n_treatments))
Pi1_test = (X_test > 0) * 1.
# returns estimate of γ/2 + δ/√(2π)
policy_effect = np.mean(est.effect(Pi0_test, Pi1_test, X_test))

In [ ]:
data = dowhy.datasets.linear_dataset(beta=10,
        num_common_causes=5, 
        num_instruments = 2,
        num_samples=10000,
        treatment_is_binary=True)
df = data["df"]
df.head()

In [ ]:
from econml.dml import DMLCateEstimator
from sklearn.linear_model import LassoCV

est = DMLCateEstimator(model_y=LassoCV(), model_t=LassoCV())
est.fit(df["y"], df["v"], df["Z0"], df[["X1", "X2"]]) # W -> high-dimensional confounders, X -> features
treatment_effects = est.effect(X_test)